In [72]:
import pysam
import pandas as pd
from tqdm import tqdm
import pandas as pd
import numpy as np

In [60]:
SF = pysam.Samfile("data/rd/SRR.sam", "rb")

In [67]:
n_skip=0
len_chr={}
with open("data/rd/SRR.sam", 'r') as f:
    for line in f:
        l = line.replace('\n', '').split('\t')
        sn = l[1].replace('SN:', '')
        ln = l[2].replace('LN:', '')
        len_chr[sn] = ln
        if line[:1] == '@':
            n_skip+=1
        else:
            break

In [66]:
sam_list = []
for sam in SF.fetch():
    sam_list.append([SF.getrname(sam.rname), sam.qname, sam.flag, sam.pos, sam.cigar, sam.seq])

In [68]:
SF.close()

In [69]:
df_sam = pd.DataFrame(sam_list, columns=['rname', 'qname', 'flag', 'pos', 'cigar', 'seq'])

In [70]:
df_sam.head()

,rname,qname,flag,pos,cigar,seq
0,chr1,SRR590764.1,147,182976,"[(0, 76)]",TGGGATCTCTGAGCTCAACAAGCCCTCTCTGGGTGGTAGGTGCAGA...
1,chr1,SRR590764.2,83,13026,"[(0, 76)]",CATGAAGGCTGTCAACCAGTCCATAGGCAAGCCTGGCTGCCTCCAG...
2,chr1,SRR590764.2,163,12963,"[(0, 76)]",TTCACTCCCAGCTCAGAGCCCAGGCCAGGGGCCCCCAAGAAAGGCT...
3,chr1,SRR590764.3,99,183736,"[(0, 76)]",AAGGCAGGGCCATCAGGCACCAAAGGGATTCTGCCAGCATAGTGCT...
4,chr1,SRR590764.3,147,183756,"[(0, 76)]",CAAAGGGATTCTGCCAGCATAGTGCTCCTGGACCAGTGATACACCC...


In [73]:
df_sam['ln'] = df_sam['rname'].apply(lambda x: int(len_chr[x]) if x in len_chr else np.nan )